Data shows that the best neighborhood is between mainly Bloomfield and Squirrel Hill South, with Bloomfield having 3 more Bike Accessible roads. 

Had to manually figure out the neighborhoods for these entries:
Resevoir - Highland Park
Highland - East Liberty
Thomas - Point Breeze
Brereton - Polish Hill
Braddock - Point Breeze/Squirrel Hills South
Sennot st - Central Oakland
Highland - East Liberty
O'Hara - Central Oakland
Allegheny circle - Central Allegheny 
3rd Ave - Central Business District 
South Side Neighborway - South Side
Lawranceville Neighborway - Central Lawranceville
Marshall - Marshall-Shadeland/Brighton Heights 
Bloomfield Friendship Neighborway - Bloomfield
Euclid Neighborway - East Liberty
Hazelwood Gold Neighborway - Hazelwood 
Reynolds street - Point Breeze 
California - California-Kirkbride
Federal - North Shore

In [ ]:
## import pandas as pd
import pandas as pd
import requests
import time

# Manually added neighborhoods (all lowercase)
manual_neighborhoods = {
    "resevoir dr": "Highland Park",
    "highland ave": "East Liberty",
    "thomas": "Point Breeze",
    "brereton/28th": "Polish Hill",
    "braddock ave": "Point Breeze",
    "braddock ave": "Squirrel Hill South",
    "sennot st": "Central Oakland",
    "highland": "East Liberty",
    "o'hara bigelow bayard": "Central Oakland",
    "allegheny circle": "Central Allegheny",
    "3rd ave": "Central Business District",
    "south side neighborway": "South Side",
    "central lawrenceville neighborway": "Central Lawrenceville",
    "marshall": "Marshall-Shadeland",
    "marshall": "Brighton Heights",
    "bloomfield-friendship neighborway": "Bloomfield",
    "euclid neighborway": "East Liberty",
    "melwood gold neighborway": "Hazelwood",
    "reynolds neighborway": "Point Breeze",
    "california": "California-Kirkbride",
    "federal": "North Shore"
}

# Function to query the OSM
def get_pittsburgh_neighborhood(query):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": f"{query}, Pittsburgh, PA",
        "format": "json",
        "addressdetails": 1,
        "limit": 1
    }
    headers = {
        "User-Agent": "BicycleLaneNeighborhoodFinder/1.0 (rgf21@pitt.edu)"
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        data = response.json()
        if not data:
            return None

        address = data[0].get("address", {})
        return (
            address.get("neighbourhood") or 
            address.get("suburb") or 
            address.get("city_district")
        )
    except Exception as e:
        print(f"API error for {query}: {e}")
        return None

# Loads the data file
df = pd.read_csv("Bicycle pavement markings, 1980 to present - Existing.tsv", sep="\t")

# cleans up file
df = df[df["Street"].notna()]
df["Street"] = df["Street"].str.strip()
df["Cross Streets"] = df["Cross Streets"].fillna("").str.strip()

# Stores results
neighborhoods = []

# Goes row by row
for idx, row in df.iterrows():
    street = row["Street"]
    cross = row["Cross Streets"]

    # Lowercase keys for manual lookup
    key = street.lower().strip()
    if key in manual_neighborhoods:
        neighborhood = manual_neighborhoods[key]
        print(f"[Manual] {street} -> {neighborhood}")
        neighborhoods.append((neighborhood, street))
        continue

    # Checks for valid cross streets
    bad_terms = {"loop", "various", "through", "park", "trail", "path"}
    use_cross = cross and all(term not in cross.lower() for term in bad_terms)

    # Simplifies cross streets
    if use_cross:
        if " to " in cross:
            cross_simple = cross.split(" to ")[0]
        elif "/" in cross:
            cross_simple = cross.split("/")[0]
        else:
            cross_simple = cross
        cross_simple = cross_simple.strip()
        query = f"{street} and {cross_simple}"
    else:
        query = street

    print(f"Querying: {query}")
    neighborhood = get_pittsburgh_neighborhood(query)

    # Fallback in case of failiure
    if not neighborhood and use_cross:
        print(f"Fallback to just street: {street}")
        neighborhood = get_pittsburgh_neighborhood(street)

    if neighborhood:
        print(f"{query} -> {neighborhood}")
        neighborhoods.append((neighborhood, street))

    time.sleep(1)

# Creates summary of data
summary_df = pd.DataFrame(neighborhoods, columns=["Neighborhood", "Street"])
summary = summary_df.groupby("Neighborhood").size().reset_index(name="Bike Lanes")
summary.to_csv("neighborhood_bike_lane_counts.csv", index=False)

print("\n Done!")
print("Neighborhood summary -> 'neighborhood_bike_lane_counts.csv'")

In [3]:
import pandas as pd

In [4]:
summary_loaded = pd.read_csv("neighborhood_bike_lane_counts.csv")
print("\nNeighborhood Bike Lane Counts:")
print(summary_loaded.sort_values(by="Bike Lanes", ascending=False).to_string(index=False))


Neighborhood Bike Lane Counts:
                    Neighborhood  Bike Lanes
                      Bloomfield          16
             Squirrel Hill South          14
                    East Liberty           8
                     Perry North           7
                   Highland Park           6
             Lower Lawrenceville           6
                       Shadyside           5
                   South Oakland           5
                         Oakland           5
              Marshall-Shadeland           4
                    Point Breeze           4
               Cultural District           3
                  East Allegheny           3
                     North Shore           3
                Allegheny Center           3
Schenley Farms Historic District           3
                     Polish Hill           3
                South Side Flats           2
                        Homewood           2
                       Hazelwood           2
                       